In [3]:
import os
import pickle
import nltk
import string
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM,Embedding
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3" #If you work on server with other people don't forget to limit your resources

In [8]:
path_to_weights = '../models/weights/'

In [11]:
dialogs = pickle.load(open('../data/dialogs.pkl', 'rb'))
scenery = pickle.load(open('../data/scenery.pkl', 'rb'))

In [12]:
def average_lenght(lines):
    return np.mean([len(line) for line in lines])

In [13]:
dialog_lenght = int(average_lenght(dialogs)) #average length of dialog
scenery_lenght = int(average_lenght(scenery)) #average length of scenery description

In [14]:
def make_sequence(tokens, seuqence_len):
    sequences = list()
    for i in range(seuqence_len, len(tokens)):
        # select sequence of tokens
        seq = tokens[i-seuqence_len:i]
        # convert into a line
        line = ' '.join(seq)
        # store
        sequences.append(line)
    return sequences

In [15]:
def join(lines):
    res = []
    for line in lines:
        res += line
    return res

In [16]:
dialog_sequences = make_sequence(join(dialogs), int(dialog_lenght))
scenery_sequences = make_sequence(join(scenery), int(scenery_lenght))

In [40]:
#save sequences
pickle.dump(dialog_sequences, open('../data/dialogs_sequences.pkl', 'wb'))
pickle.dump(scenery_sequences, open('../data/scenery_sequences.pkl', 'wb'))

In [25]:
def build_model(vocab_size, sequence_length, embedding_size, lstm_size, dence_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=sequence_length))
    model.add(LSTM(lstm_size, return_sequences=True))
    model.add(LSTM(lstm_size))
    model.add(Dense(dence_size, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    return model

In [31]:
def create_text_generator(name,lines, expected_sequence_length, embedding_size=50, lstm_size=100, dence_size=100):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    vocab_size = len(tokenizer.word_index) + 1
    print("Vocablulary size: ", vocab_size)
    
    sequences = tokenizer.texts_to_sequences(lines)
    sequences = [x for x in sequences if len(x) == expected_sequence_length]
    sequences = np.array(sequences)
    
    X, y = sequences[:,:-1], sequences[:,-1]
    y = to_categorical(y, num_classes=vocab_size)
    seq_length = X.shape[1]
    
    model = build_model(vocab_size, seq_length, embedding_size, lstm_size, dence_size)
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit model
    mc = keras.callbacks.ModelCheckpoint(path_to_weights + name + '_weights{epoch:08d}.h5', 
                                     save_weights_only=True, period=50)
    history = model.fit(X, y, batch_size=128, callbacks=[mc], epochs=100)
    return tokenizer, history, model
    

In [29]:
dialog_tokenizer, dialog_train_history, dialog_model = create_text_generator('dialog', dialog_sequences, dialog_lenght)

Vocablulary size:  7630
Epoch 1/500
170612/170612 [==============================] - 77s 450us/step - loss: 6.6937 - acc: 0.0441
Epoch 2/500
170612/170612 [==============================] - 76s 443us/step - loss: 6.3149 - acc: 0.0629
Epoch 3/500
170612/170612 [==============================] - 74s 436us/step - loss: 6.0810 - acc: 0.0812
Epoch 4/500
170612/170612 [==============================] - 74s 436us/step - loss: 5.8922 - acc: 0.0901
Epoch 5/500
170612/170612 [==============================] - 75s 438us/step - loss: 5.7117 - acc: 0.0973
Epoch 6/500
170612/170612 [==============================] - 74s 435us/step - loss: 5.5430 - acc: 0.1039
Epoch 7/500
170612/170612 [==============================] - 75s 439us/step - loss: 5.4014 - acc: 0.1083
Epoch 8/500
170612/170612 [==============================] - 74s 434us/step - loss: 5.2661 - acc: 0.1137
Epoch 9/500
170612/170612 [==============================] - 75s 439us/step - loss: 5.1417 - acc: 0.1178
Epoch 10/500
170612/170612 [===

170612/170612 [==============================] - 75s 441us/step - loss: 1.9752 - acc: 0.5527
Epoch 79/500
170612/170612 [==============================] - 75s 440us/step - loss: 1.9524 - acc: 0.5574
Epoch 80/500
170612/170612 [==============================] - 75s 441us/step - loss: 1.9278 - acc: 0.5623
Epoch 81/500
170612/170612 [==============================] - 75s 437us/step - loss: 1.9080 - acc: 0.5667
Epoch 82/500
170612/170612 [==============================] - 74s 436us/step - loss: 1.8828 - acc: 0.5722
Epoch 83/500
170612/170612 [==============================] - 75s 439us/step - loss: 1.8649 - acc: 0.5748
Epoch 84/500
170612/170612 [==============================] - 75s 440us/step - loss: 1.8470 - acc: 0.5791
Epoch 85/500
170612/170612 [==============================] - 75s 439us/step - loss: 1.8266 - acc: 0.5844
Epoch 86/500
170612/170612 [==============================] - 75s 442us/step - loss: 1.8045 - acc: 0.5874
Epoch 87/500
170612/170612 [==============================]

170612/170612 [==============================] - 93s 543us/step - loss: 1.0634 - acc: 0.7456
Epoch 155/500
170612/170612 [==============================] - 93s 545us/step - loss: 1.0493 - acc: 0.7493
Epoch 156/500
170612/170612 [==============================] - 94s 551us/step - loss: 1.0391 - acc: 0.7509
Epoch 157/500
170612/170612 [==============================] - 94s 551us/step - loss: 1.0207 - acc: 0.7553
Epoch 158/500
170612/170612 [==============================] - 93s 545us/step - loss: 1.0466 - acc: 0.7474
Epoch 159/500
170612/170612 [==============================] - 96s 565us/step - loss: 1.0248 - acc: 0.7543
Epoch 160/500
170612/170612 [==============================] - 95s 558us/step - loss: 1.0164 - acc: 0.7558
Epoch 161/500
170612/170612 [==============================] - 94s 550us/step - loss: 1.0122 - acc: 0.7570
Epoch 162/500
170612/170612 [==============================] - 93s 546us/step - loss: 1.0098 - acc: 0.7573
Epoch 163/500
170612/170612 [======================

170612/170612 [==============================] - 85s 501us/step - loss: 0.7808 - acc: 0.8067
Epoch 231/500
170612/170612 [==============================] - 88s 518us/step - loss: 0.7770 - acc: 0.8067
Epoch 232/500
170612/170612 [==============================] - 85s 495us/step - loss: 0.7676 - acc: 0.8104
Epoch 233/500
170612/170612 [==============================] - 85s 496us/step - loss: 0.7718 - acc: 0.8086
Epoch 234/500
170612/170612 [==============================] - 85s 499us/step - loss: 0.7693 - acc: 0.8089
Epoch 235/500
170612/170612 [==============================] - 84s 495us/step - loss: 0.7487 - acc: 0.8146
Epoch 236/500
170612/170612 [==============================] - 85s 501us/step - loss: 0.7703 - acc: 0.8087
Epoch 237/500
170612/170612 [==============================] - 85s 499us/step - loss: 0.7674 - acc: 0.8095
Epoch 238/500
170612/170612 [==============================] - 86s 503us/step - loss: 0.7508 - acc: 0.8138
Epoch 239/500
170612/170612 [======================

170612/170612 [==============================] - 82s 479us/step - loss: 0.6443 - acc: 0.8376
Epoch 307/500
170612/170612 [==============================] - 84s 490us/step - loss: 0.6718 - acc: 0.8301
Epoch 308/500
170612/170612 [==============================] - 81s 476us/step - loss: 0.6240 - acc: 0.8433
Epoch 309/500
170612/170612 [==============================] - 82s 481us/step - loss: 0.6390 - acc: 0.8390
Epoch 310/500
170612/170612 [==============================] - 82s 478us/step - loss: 0.6411 - acc: 0.8383
Epoch 311/500
170612/170612 [==============================] - 80s 469us/step - loss: 0.6462 - acc: 0.8368
Epoch 312/500
170612/170612 [==============================] - 81s 476us/step - loss: 0.6581 - acc: 0.8336
Epoch 313/500
170612/170612 [==============================] - 81s 476us/step - loss: 0.6320 - acc: 0.8404
Epoch 314/500
170612/170612 [==============================] - 82s 481us/step - loss: 0.6466 - acc: 0.8357
Epoch 315/500
170612/170612 [======================

170612/170612 [==============================] - 87s 508us/step - loss: 0.5764 - acc: 0.8521
Epoch 383/500
170612/170612 [==============================] - 85s 496us/step - loss: 0.5853 - acc: 0.8497
Epoch 384/500
170612/170612 [==============================] - 86s 504us/step - loss: 0.5714 - acc: 0.8535
Epoch 385/500
170612/170612 [==============================] - 86s 504us/step - loss: 0.5635 - acc: 0.8559
Epoch 386/500
170612/170612 [==============================] - 82s 482us/step - loss: 0.5735 - acc: 0.8526
Epoch 387/500
170612/170612 [==============================] - 80s 471us/step - loss: 0.5742 - acc: 0.8538
Epoch 388/500
170612/170612 [==============================] - 84s 495us/step - loss: 0.5873 - acc: 0.8501
Epoch 389/500
170612/170612 [==============================] - 83s 489us/step - loss: 0.5968 - acc: 0.8472
Epoch 390/500
170612/170612 [==============================] - 84s 495us/step - loss: 0.5593 - acc: 0.8571
Epoch 391/500
170612/170612 [======================

170612/170612 [==============================] - 82s 481us/step - loss: 0.5251 - acc: 0.8649
Epoch 459/500
170612/170612 [==============================] - 87s 513us/step - loss: 0.5244 - acc: 0.8652
Epoch 460/500
170612/170612 [==============================] - 82s 481us/step - loss: 0.5385 - acc: 0.8600
Epoch 461/500
170612/170612 [==============================] - 81s 474us/step - loss: 0.5254 - acc: 0.8640
Epoch 462/500
170612/170612 [==============================] - 84s 495us/step - loss: 0.5246 - acc: 0.8645
Epoch 463/500
170612/170612 [==============================] - 86s 504us/step - loss: 0.5235 - acc: 0.8645
Epoch 464/500
170612/170612 [==============================] - 85s 500us/step - loss: 0.5252 - acc: 0.8629
Epoch 465/500
170612/170612 [==============================] - 85s 499us/step - loss: 0.5326 - acc: 0.8624
Epoch 466/500
170612/170612 [==============================] - 85s 498us/step - loss: 0.5241 - acc: 0.8640
Epoch 467/500
170612/170612 [======================

In [32]:
scenery_tokenizer, scenery_train_history, scenery_model = create_text_generator('scenery', scenery_sequences, scenery_lenght)

Vocablulary size:  9996
Epoch 1/100
241569/241569 [==============================] - 430s 2ms/step - loss: 6.8407 - acc: 0.0780
Epoch 2/100
241569/241569 [==============================] - 428s 2ms/step - loss: 6.3787 - acc: 0.0909
Epoch 3/100
241569/241569 [==============================] - 431s 2ms/step - loss: 6.0014 - acc: 0.1123
Epoch 4/100
241569/241569 [==============================] - 468s 2ms/step - loss: 5.6901 - acc: 0.1275
Epoch 5/100
241569/241569 [==============================] - 420s 2ms/step - loss: 5.4337 - acc: 0.1391
Epoch 6/100
241569/241569 [==============================] - 445s 2ms/step - loss: 5.2070 - acc: 0.1489
Epoch 7/100
241569/241569 [==============================] - 428s 2ms/step - loss: 5.0020 - acc: 0.1566
Epoch 8/100
241569/241569 [==============================] - 419s 2ms/step - loss: 4.8111 - acc: 0.1645
Epoch 9/100
241569/241569 [==============================] - 422s 2ms/step - loss: 4.6323 - acc: 0.1724
Epoch 10/100
241569/241569 [============

241569/241569 [==============================] - 468s 2ms/step - loss: 1.4236 - acc: 0.6542
Epoch 79/100
241569/241569 [==============================] - 448s 2ms/step - loss: 1.4079 - acc: 0.6586
Epoch 80/100
241569/241569 [==============================] - 439s 2ms/step - loss: 1.3875 - acc: 0.6630
Epoch 81/100
241569/241569 [==============================] - 440s 2ms/step - loss: 1.3734 - acc: 0.6656
Epoch 82/100
241569/241569 [==============================] - 441s 2ms/step - loss: 1.3595 - acc: 0.6679
Epoch 83/100
241569/241569 [==============================] - 434s 2ms/step - loss: 1.3404 - acc: 0.6723
Epoch 84/100
241569/241569 [==============================] - 424s 2ms/step - loss: 1.3247 - acc: 0.6755
Epoch 85/100
241569/241569 [==============================] - 426s 2ms/step - loss: 1.3112 - acc: 0.6786
Epoch 86/100
241569/241569 [==============================] - 423s 2ms/step - loss: 1.2955 - acc: 0.6816
Epoch 87/100
241569/241569 [==============================] - 423s 2

In [39]:
#save models
pickle.dump(dialog_model, open('../models/dialog_model.pkl', 'wb'))
pickle.dump(scenery_model, open('../models/scenery_model.pkl', 'wb'))

In [33]:
#save tokenizers
pickle.dump(dialog_tokenizer, open('../models/dialog_tokenizer.pkl', 'wb'))
pickle.dump(scenery_tokenizer, open('../models/scenery_tokenizer.pkl', 'wb'))